In [ ]:
from tqdm import tqdm
import pandas as pd
import ast
import pickle

with open("side_bar_reviews_list", "rb") as fp1:
  side_bar_reviews_list = pickle.load(fp1)
with open("main_bar_reviews_list", "rb") as fp2:
  main_bar_reviews_list = pickle.load(fp2)
with open("recommendation_id_updated", "rb") as fp3:
  recommendation_id = pickle.load(fp3)
reviews_list=pd.read_csv("FPS_reviews.csv")

# Extracting control IDs

How it works: we simply take review IDs that were not visible in either the recently-sorted (unobservable) reviews or the default-ordered (observable) reviews, we then added it to our list of control IDs for comparision 

In [ ]:
recommendation_id = [str(elem) for elem in tqdm(recommendation_id)]
main_bar_reviews_list=[[str(elem) for elem in inner_list] for (inner_list) in tqdm(main_bar_reviews_list)]
side_bar_reviews_list=[[str(elem) for elem in inner_list] for (inner_list) in tqdm(side_bar_reviews_list)]

100%|██████████| 920536/920536 [00:16<00:00, 55195.50it/s] 


In [ ]:
reviews_list['recommendation_id'] = reviews_list['recommendation_id'].astype(str)
reviews_list['game_id'] = reviews_list['game_id'].astype(str)
reviews_list["timestamp_created"] = reviews_list["timestamp_created"].astype(int)
reviews_list

,index,recommendation_id,game_id,score,steam_id,review,timestamp_created,timestamp_updated,votes_up,order_of_visible_comments,order_of_scores,order_of_times
0,0,117084240,1533420,0.934666,76561197990540569,"if you like those crazy ass Japanese movies, M...",1655402060,1655402060,247,"['117083497', '117083452', '117080996', '11708...","[0.8972995281219482, 0.8121684193611145, 0.728...","[1655401168, 1655401101, 1655397878, 165539730..."
1,1,117083497,1533420,0.897300,76561198043707357,Purchased a Machine Girl album. got a free gam...,1655401168,1655401168,152,"['117083452', '117080996', '117080505', '11708...","[0.8121684193611145, 0.7286536693572998, 0.695...","[1655401101, 1655397878, 1655397302, 165540099..."
2,2,117089901,1533420,0.867490,76561198203123848,a gore-obsessed chick in a bunny mask called m...,1655409089,1655409089,92,"['117084240', '117083497', '117083452', '11708...","[0.9346663355827332, 0.8972995281219482, 0.812...","[1655402060, 1655401168, 1655401101, 165540230..."
3,3,117083452,1533420,0.812168,76561198097251074,Stuck with a black screen after the intro. Int...,1655401101,1655401101,68,"['117080996', '117080505', '117083370', '11708...","[0.7286536693572998, 0.6956068277359008, 0.667...","[1655397878, 1655397302, 1655400992, 165539716..."
4,4,117084445,1533420,0.757732,76561198157693142,machine girl <3,1655402308,1655402308,40,"['117084240', '117083497', '117083452', '11708...","[0.9346663355827332, 0.8972995281219482, 0.812...","[1655402060, 1655401168, 1655401101, 165539787..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1056935,1056935,113604297,1445120,0.000000,76561198271143660,reviews really werent kidding about the font b...,1649544470,1649544470,0,"['85056140', '78634674', '78728813', '81747692...","[0.7444049715995787, 0.6891525983810424, 0.603...","[1611180708, 1604382962, 1604523185, 160728858..."
1056936,1056936,111673657,1445120,0.000000,76561198836980654,"Dear Developers,\n\nIve played this game for 1...",1646570504,1646570504,0,"['85056140', '78634674', '78728813', '81747692...","[0.7444049715995787, 0.6891525983810424, 0.603...","[1611180708, 1604382962, 1604523185, 160728858..."
1056937,1056937,90986327,1445120,0.000000,76561198283885247,"The font used doesn't help a lot, Hs and Ks as...",1619497434,1619497499,0,"['85056140', '78634674', '78728813', '81747692...","[0.7444049715995787, 0.6891525983810424, 0.603...","[1611180708, 1604382962, 1604523185, 160728858..."
1056938,1056938,90049741,1445120,0.000000,76561198026846672,font is horrible.\nToo fast for most people\nB...,1618038505,1618038505,1,['85056140'],[0.7444049715995787],[1611180708]


In [ ]:
import pandas as pd

filtered_recommendation_ids = []
for i in (range(len(recommendation_id))):
  # Parse out the rows where recommendation_id[i] == reviews_list["recommendation_id"]
  target_rows = (reviews_list[reviews_list["recommendation_id"] == recommendation_id[i]]).reset_index(drop=True)

  # Get the values from columns "timestamp_created" and "game_id"
  time_id = target_rows["timestamp_created"].values[0]
  
  # Find the corresponding value in column "game_id"
  game_ids = target_rows["game_id"].values
  
  # If there are more than 1 corresponding values, select the one that does not begin with '['
  if len(game_ids) > 1:
      game_id = [game_id for game_id in game_ids if not str(game_id).startswith('[')][0]
  else:
      game_id = game_ids[0]

  # Select rows with the same game_id
  temp = (reviews_list[reviews_list["game_id"] == game_id]).reset_index(drop=True)

  # Remove rows where the values in main_bar_reviews_list[i] and side_bar_reviews_list[i] are present in temp["recommendation_id"]
  temp = (temp[~temp["recommendation_id"].isin(main_bar_reviews_list[i] + side_bar_reviews_list[i])]).reset_index(drop=True)

  # Sort by temp["timestamp_created"]
  temp.sort_values(by="timestamp_created", ascending=False, inplace=True)

  # Filter rows by time range
  temp = (temp[(temp["timestamp_created"] <= time_id) & (temp["timestamp_created"] > time_id - (86400 * 30))]).reset_index(drop=True)

  # Remove the existing recommendation_id[i] from the dataframe
  temp = (temp[temp["recommendation_id"] != recommendation_id[i]]).reset_index(drop=True)

  # Extract recommendation_ids and append to filtered_recommendation_ids
  filtered_recommendation_ids.append(temp["recommendation_id"].tolist())

In [ ]:
with open('control_recommendation_ids_all.pkl', 'wb') as f:
  # Dump the list to the file using the pickle module
  pickle.dump(filtered_recommendation_ids, f)